In [1]:
import numpy as np
import h5py
import os
from collections import OrderedDict
import matplotlib.pyplot as plt
import time
from skimage import img_as_float
from torch import nn, FloatTensor
import torch
from torchvision.utils import make_grid
import sys
import copy
import importlib

In [6]:
sys.path.append("/home/ziniuw/thesis-yimeng-v2")
from thesis_v2.data.prepared.yuanyuan_8k import get_data
from util.get_data_NS import reorginize_data
from models_8K.model import FKCNN_2l, FKCNN_3l
data = get_data('a', 256, 64, ['042318'], read_only=False, scale=0.5)
data_Sep = reorginize_data(data) #train model one at a time
weight = np.load("/home/ziniuw/Tangdata/filter_79.npy")
gabor = np.load("/home/ziniuw/Tangdata/gabor.npy")
gabor = gabor.reshape(24,1,10,10)
weight = weight.reshape(79,1,9,9)
weight = weight[[0,3,5,8,9,11,14,17,18,20,23,25,27,32,36,37,40,44,53,57,58,64,65,74],:,:,:]
temp = np.load("/home/ziniuw/Tangdata/weight_64.npy")
temp = temp.reshape(64,1,9,9)

In [3]:
#receptive field location
RF = [0]*29
RF[0] = (np.arange(40,60), np.arange(38,58))
RF[1] = (np.arange(22,42), np.arange(22,42))
RF[2] = (np.arange(22,42), np.arange(22,42))
RF[3] = (np.arange(44,64), np.arange(22,42))
RF[4] = (np.arange(40,60), np.arange(38,58))
RF[5] = (np.arange(35,55), np.arange(32,52))
RF[6] = (np.arange(28,48), np.arange(40,60))
RF[7] = (np.arange(28,48), np.arange(35,55))
RF[8] = (np.arange(28,48), np.arange(28,48))
RF[9] = (np.arange(28,48), np.arange(28,48))
RF[10] = (np.arange(28,48), np.arange(28,48))
RF[11] = (np.arange(22,42), np.arange(22,42))
RF[12] = (np.arange(22,42), np.arange(22,42))
RF[13] = (np.arange(22,42), np.arange(22,42))
RF[14] = (np.arange(22,42), np.arange(22,42))
RF[15] = (np.arange(22,42), np.arange(22,42))
RF[16] = (np.arange(22,42), np.arange(22,42))
RF[17] = (np.arange(22,42), np.arange(22,42))
RF[18] = (np.arange(22,42), np.arange(22,42))
RF[19] = (np.arange(22,42), np.arange(22,42))
RF[20] = (np.arange(40,60), np.arange(22,42))
RF[21] = (np.arange(22,42), np.arange(22,42))
RF[22] = (np.arange(22,42), np.arange(22,42))
RF[23] = (np.arange(22,42), np.arange(22,42))
RF[24] = (np.arange(40,60), np.arange(22,42))
RF[25] = (np.arange(40,60), np.arange(22,42))
RF[26] = (np.arange(40,60), np.arange(38,58))
RF[27] = (np.arange(22,42), np.arange(22,42))
RF[28] = (np.arange(40,60), np.arange(22,42))

In [7]:
def data_preprocess(data, RF):
    new_data = dict()
    for i in data:
        X = data[i][0][:,:,RF[i][0],:]
        train_X = X[:,:,:,RF[i][1]]
        X = data[i][2][:,:,RF[i][0],:]
        val_X = X[:,:,:,RF[i][1]]
        X = data[i][4][:,:,RF[i][0],:]
        test_X = X[:,:,:,RF[i][1]]
        new_data[i] = (train_X,data[i][1],val_X,data[i][3],test_X,data[i][5])
    return new_data
data_Sep = data_preprocess(data_Sep, RF)

In [8]:
print(len(data_Sep))
print(len(data_Sep[0]))

29
6


In [23]:
corr_PPR = []
sys.path.append('/home/ziniuw/Fixed_Kernel_CNN/run_script')
from run_PPR import PPR,train_one
for i in data_Sep:
    print(i)
    model = PPR(nlayers=1)
    res = train_one(model, data_Sep[i], show_every = 100)
    print(res[1])
    corr_PPR.append(res[1])

0
Epoch 100
-0.029682096
0.13301328
1
Epoch 100
-0.08389391
0.08725591
2
Epoch 100
-0.08102161
0.21818666
3
Epoch 100
-0.0070990035
0.09710656
4
Epoch 100
0.0038746698
0.20066239
5
Epoch 100
0.0024421706
0.21626715
6
Epoch 100
-0.0825259
0.12287873
7
Epoch 100
-0.105146326
0.21522483
8
Epoch 100
-0.24017942
0.37717316
9
Epoch 100
-0.41233647
0.46845764
10
Epoch 100
-0.11197203
0.31706822
11
Epoch 100
-0.17356566
0.2112189
12
Epoch 100
-0.18762358
0.3297438
13
Epoch 100
-0.41637382
0.34965792
14
Epoch 100
0.054071676
0.15557271
15
Epoch 100
-0.17453505
0.28013924
16
Epoch 100
-0.014443121
0.06116931
17
Epoch 100
-0.07187479
0.25006074
18
Epoch 100
-0.07639228
0.28682074
19
Epoch 100
-0.18180405
0.3219115
20
Epoch 100
-0.059521586
0.037420303
21
Epoch 100
-0.03739059
0.18490668
22
Epoch 100
-0.086003534
0.03546903
23
Epoch 100
-0.0071701617
0.0481134
24
Epoch 100
-0.06865594
0.043197475
25
Epoch 100
-0.018806284
0.041585565
26
Epoch 100
-0.0668305
0.051816028
27
Epoch 100
0.037220124
-0.

In [24]:
corr_CPPR = []
from run_CPPR import CPPR,train_one
for i in data_Sep:
    print(i)
    model = CPPR(nlayers=5)
    res = train_one(model, data_Sep[i], show_every = 100)
    print(res[1])
    corr_CPPR.append(res[1])

0
Epoch 100
0.12730853
0.15682401
1
Epoch 100
-0.03037788
0.15784436
2
Epoch 100
0.08880636
0.1741711
3
Epoch 100
0.03127986
0.13541563
4
Epoch 100
0.15664425
0.14344
5
Epoch 100
0.050199695
0.31020147
6
Epoch 100
0.02000505
-0.075975545
7
Epoch 100
0.13866393
0.23963635
8
Epoch 100
0.19874725
-0.25708437
9
Epoch 100
0.027587349
0.056530852
10
Epoch 100
0.25700724
0.26871213
11
Epoch 100
0.14302896
0.18658453
12
Epoch 100
0.19576645
0.26728493
13
Epoch 100
-0.16856892
0.33489805
14
Epoch 100
0.15581146
0.0739963
15
Epoch 100
-0.038142696
0.2140441
16
Epoch 100
0.15792468
-0.031265814
17
Epoch 100
0.17690769
0.23532772
18
Epoch 100
0.04512353
0.3124229
19
Epoch 100
0.016158732
0.16192132
20
Epoch 100
0.16253693
0.04922514
21
Epoch 100
0.048287753
0.2453812
22
Epoch 100
-0.06285413


TypeError: 'NoneType' object is not subscriptable